# Analysis of the Evian water

The tutorial considers the problem dedicated to checking the quality of the Evian water using Reaktoro with custom-defined chemical composition.

|![Evian water label](../../images/evian-chemical-water-composition.png)|
|:--:|
|Evian water label, Source: wikipedia.org|

We start from setting a chemical system, constraints of temperature, pressure, and pH, and their values:

In [ ]:
from reaktoro import *
from math import *

db = PhreeqcDatabase("phreeqc.dat")

# Create an aqueous phase automatically selecting all species with provided elements
aqueousphase = AqueousPhase(speciate("C Ca Cl H K Mg N Na O S Si"))
aqueousphase.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2"),
))

gaseousphase = GaseousPhase("CO2(g)")
gaseousphase.setActivityModel(ActivityModelPengRobinson())

minerals = MineralPhases()

# Create the chemical system
system = ChemicalSystem(db, aqueousphase, gaseousphase, minerals)

specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.pH()

solver = EquilibriumSolver(specs)

conditions = EquilibriumConditions(specs)
conditions.temperature(25.0, "celsius")
conditions.pressure(1.0, "atm")
conditions.pH(7.2)

The composition of the initial chemical state is taken from the plot with a label of Evian water. However, since the concentrations of the species in Evian water are defined as milligrams per liter (mg/L), we need to convert these values to moles using species molar masses. The latter can be obtained using function `db.species("H+").molarMass()`. Also, to convert `mg` into `kg`, we multiply the values by 1e-6. We also equilibrate the given chemical state.

In [ ]:
state = ChemicalState(system)

state.set("H2O"    , 1.0, "kg")
state.set("Ca+2"   , 80 * 1e-6   / db.species("Ca+2").molarMass()   , "mol")  # 80 * 1e-6 kg / (MW(Ca++) kg / mol)
state.set("Cl-"    , 6.8 * 1e-6  / db.species("Cl-").molarMass()    , "mol")  # 6.8 * 1e-6 kg / (MW(Cl-) kg / mol)
state.set("HCO3-"  , 350 * 1e-6  / db.species("HCO3-").molarMass()  , "mol")  # 350 * 1e-6 kg / (MW(HCO3-) kg / mol)
state.set("Mg+2"   , 26 * 1e-6   / db.species("Mg+2").molarMass()   , "mol")  # 26 * 1e-6 kg / (MW(Mg++) kg / mol)
state.set("NO3-"   , 3.7 * 1e-6  / db.species("NO3-").molarMass()   , "mol")  # 3.7 * 1e-6 kg / (MW(NO3-) kg / mol)
state.set("K+"     , 1 * 1e-6    / db.species("K+").molarMass()     , "mol")  # 1 * 1e-6 kg / (MW(K+) kg / mol)
state.set("Na+"    , 6.5 * 1e-6  / db.species("Na+").molarMass()    , "mol")  # 6.5 * 1e-6 kg / (MW(Na+) kg / mol)
state.set("SO4-2"  , 12.6 * 1e-6 / db.species("SO4-2").molarMass()  , "mol")  # 12.6 * 1e-6 kg / (MW(SO4--) kg / mol)
state.set("SiO2(a)", 15 * 1e-6   / db.species("SiO2(a)").molarMass(), "mol")  # 15 * 1e-6 kg / (MW(SiO2) kg/ mol

res = solver.solve(state, conditions)

Next, we perform an analysis of the equilibrated chemical state of the Evian water:
* evaluate saturation indices of all minerals in the database that can be formed by chemical reactions
* analyse solution composition

To obtain saturation indices of the minerals, we need to access the aqueous properties of the calculated chemical state. The saturation index is defined as a log10 of the ratio of equilibrium constant and reaction quotient. It is 0 for minerals that are *saturated* or in other words *precipitated and are staying in equilibrium with the solution*, SI > 0 for *supersaturated minerals*, and SI < 0 for *undersaturated minerals*.

In [ ]:
# Calculate saturation indices
props = AqueousProps(state)
print("SI (Calcite, CaCO3)       = ", props.saturationIndexLg('Calcite'))
print("SI (Aragonite, CaCO3)     = ", props.saturationIndexLg('Aragonite'))
print("SI (Dolomite, CaMg(CO3)2) = ", props.saturationIndexLg('Dolomite'))
print("SI (Quartz, SiO2)         = ", props.saturationIndexLg('Quartz'))
print("SI (Chalcedony, SiO2)     = ", props.saturationIndexLg('Chalcedony'))

Based on the results above, the water is *saturated with dolomite*, whereas calcite and quartz are *undersaturated*.

**TASK**: check if this agrees how this agrees with the amount of these minerals after in the resulting chemical state.

In [ ]:
# -------------------------------------- #
# Code cell for the task
# -------------------------------------- #